In [1]:
import sqlite3
from astropy.table import Table
import matplotlib.pyplot as plt
%matplotlib inline
from cesium import featurize
import numpy as np

In [2]:
curve = Table.read('scripts/ptf_query/byOid/curves_oid___27882090015687.tbl', format = 'ipac')
curve2 = Table.read('scripts/ptf_query/byOid/curves_oid_1000492020008501.tbl', format = 'ipac')

In [3]:
feats_to_use = ['amplitude',
                'flux_percentile_ratio_mid20', 
                'flux_percentile_ratio_mid35', 
                'flux_percentile_ratio_mid50', 
                'flux_percentile_ratio_mid65', 
                'flux_percentile_ratio_mid80', 
                'max_slope', 
                'maximum', 
                'median',
                'median_absolute_deviation', 
                'minimum',
                'percent_amplitude',
                'percent_beyond_1_std', 
                'percent_close_to_median', 
                'percent_difference_flux_percentile',
                'period_fast', 
                'qso_log_chi2_qsonu',
                'qso_log_chi2nuNULL_chi2nu',
                'skew',
                'std',
                'stetson_j',
                'stetson_k',
                'weighted_average',
                'fold2P_slope_10percentile',
                'fold2P_slope_90percentile',
                'freq1_amplitude1',
                'freq1_amplitude2',
                'freq1_amplitude3',
                'freq1_amplitude4',
                'freq1_freq',
                'freq1_lambda',
                'freq1_rel_phase2',
                'freq1_rel_phase3',
                'freq1_rel_phase4',
                'freq1_signif',
                'freq2_amplitude1',
                'freq2_amplitude2',
                'freq2_amplitude3',
                'freq2_amplitude4',
                'freq2_freq',
                'freq2_rel_phase2',
                'freq2_rel_phase3',
                'freq2_rel_phase4',
                'freq3_amplitude1',
                'freq3_amplitude2',
                'freq3_amplitude3',
                'freq3_amplitude4',
                'freq3_freq',
                'freq3_rel_phase2',
                'freq3_rel_phase3',
                'freq3_rel_phase4',
                'freq_amplitude_ratio_21',
                'freq_amplitude_ratio_31',
                'freq_frequency_ratio_21',
                'freq_frequency_ratio_31',
                'freq_model_max_delta_mags',
                'freq_model_min_delta_mags',
                'freq_model_phi1_phi2',
                'freq_n_alias',
                'freq_signif_ratio_21',
                'freq_signif_ratio_31',
                'freq_varrat',
                'freq_y_offset',
                'linear_trend',
                'medperc90_2p_p',
                'p2p_scatter_2praw',
                'p2p_scatter_over_mad',
                'p2p_scatter_pfold_over_mad',
                'p2p_ssqr_diff_over_var',
                'scatter_res_raw'
               ]
fset = featurize.featurize_time_series(curve['obsmjd'], curve['mag_autocorr'], curve['magerr_auto'],
                                       meta_features = {'oid': str(curve['oid'][0])}, features_to_use = feats_to_use)

/home/nke2/miniconda3/envs/tmpJupyter/lib/python3.6/site-packages/dask/async.py:13: UserWarning: `dask.async.get_sync` has been moved to `dask.local.get_sync`, please update your imports
  warnings.warn(_msg.format('get_sync'))


In [4]:
fset2 = featurize.featurize_time_series(curve2['obsmjd'], curve2['mag_autocorr'], curve2['magerr_auto'],
                                        meta_features = {'oid': str(curve2['oid'][0])}, features_to_use = feats_to_use)

/home/nke2/miniconda3/envs/tmpJupyter/lib/python3.6/site-packages/dask/async.py:13: UserWarning: `dask.async.get_sync` has been moved to `dask.local.get_sync`, please update your imports
  warnings.warn(_msg.format('get_sync'))


In [5]:
conn = sqlite3.connect('featurestest.db')
cur = conn.cursor()

In [ ]:
cur.execute("""create table feats2(
    amplitude,
    flux_percentile_ratio_mid20, 
    flux_percentile_ratio_mid35, 
    flux_percentile_ratio_mid50, 
    flux_percentile_ratio_mid65, 
    flux_percentile_ratio_mid80, 
    max_slope, 
    maximum, 
    median,
    median_absolute_deviation, 
    minimum, 
    percent_amplitude,
    percent_beyond_1_std, 
    percent_close_to_median, 
    percent_difference_flux_percentile,
    period_fast, 
    qso_log_chi2_qsonu,
    qso_log_chi2nuNULL_chi2nu,
    skew,
    std,
    stetson_j,
    stetson_k,
    weighted_average,
    fold2P_slope_10percentile,
    fold2P_slope_90percentile,
    freq1_amplitude1,
    freq1_amplitude2,
    freq1_amplitude3,
    freq1_amplitude4,
    freq1_freq,
    freq1_lambda,
    freq1_rel_phase2,
    freq1_rel_phase3,
    freq1_rel_phase4,
    freq1_signif,
    freq2_amplitude1,
    freq2_amplitude2,
    freq2_amplitude3,
    freq2_amplitude4,
    freq2_freq,
    freq2_rel_phase2,
    freq2_rel_phase3,
    freq2_rel_phase4,
    freq3_amplitude1,
    freq3_amplitude2,
    freq3_amplitude3,
    freq3_amplitude4,
    freq3_freq,
    freq3_rel_phase2,
    freq3_rel_phase3,
    freq3_rel_phase4,
    freq_amplitude_ratio_21,
    freq_amplitude_ratio_31,
    freq_frequency_ratio_21,
    freq_frequency_ratio_31,
    freq_model_max_delta_mags,
    freq_model_min_delta_mags,
    freq_model_phi1_phi2,
    freq_n_alias,
    freq_signif_ratio_21,
    freq_signif_ratio_31,
    freq_varrat,
    freq_y_offset,
    linear_trend,
    medperc90_2p_p,
    p2p_scatter_2praw,
    p2p_scatter_over_mad,
    p2p_scatter_pfold_over_mad,
    p2p_ssqr_diff_over_var,
    scatter_res_raw,
    oid string unique primary key
)""")

In [ ]:
cur.execute("drop table feats2")

In [ ]:
cur.execute("""insert into feats2 values (
            ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
            ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
            fset2.get_values()[0])

In [6]:
cur.execute("""PRAGMA table_info(feats2)""")
cur.fetchall()

[(0, 'oid', 'string', 0, None, 1),
 (1, 'amplitude', '', 0, None, 0),
 (2, 'flux_percentile_ratio_mid20', '', 0, None, 0),
 (3, 'flux_percentile_ratio_mid35', '', 0, None, 0),
 (4, 'flux_percentile_ratio_mid50', '', 0, None, 0),
 (5, 'flux_percentile_ratio_mid65', '', 0, None, 0),
 (6, 'flux_percentile_ratio_mid80', '', 0, None, 0),
 (7, 'max_slope', '', 0, None, 0),
 (8, 'maximum', '', 0, None, 0),
 (9, 'median', '', 0, None, 0),
 (10, 'median_absolute_deviation', '', 0, None, 0),
 (11, 'minimum', '', 0, None, 0),
 (12, 'percent_amplitude', '', 0, None, 0),
 (13, 'percent_beyond_1_std', '', 0, None, 0),
 (14, 'percent_close_to_median', '', 0, None, 0),
 (15, 'percent_difference_flux_percentile', '', 0, None, 0),
 (16, 'period_fast', '', 0, None, 0),
 (17, 'qso_log_chi2_qsonu', '', 0, None, 0),
 (18, 'qso_log_chi2nuNULL_chi2nu', '', 0, None, 0),
 (19, 'skew', '', 0, None, 0),
 (20, 'std', '', 0, None, 0),
 (21, 'stetson_j', '', 0, None, 0),
 (22, 'stetson_k', '', 0, None, 0),
 (23, 'we

In [ ]:
def saving (lc, db, tName): #pass in lightcurve table and locaiton of database
    feats_to_use = [
                'amplitude',
                'flux_percentile_ratio_mid20', 
                'flux_percentile_ratio_mid35', 
                'flux_percentile_ratio_mid50', 
                'flux_percentile_ratio_mid65', 
                'flux_percentile_ratio_mid80', 
                'max_slope', 
                'maximum', 
                'median',
                'median_absolute_deviation', 
                'minimum',
                'percent_amplitude',
                'percent_beyond_1_std', 
                'percent_close_to_median', 
                'percent_difference_flux_percentile',
                'period_fast', 
                'qso_log_chi2_qsonu',
                'qso_log_chi2nuNULL_chi2nu',
                'skew',
                'std',
                'stetson_j',
                'stetson_k',
                'weighted_average',
                'fold2P_slope_10percentile',
                'fold2P_slope_90percentile',
                'freq1_amplitude1',
                'freq1_amplitude2',
                'freq1_amplitude3',
                'freq1_amplitude4',
                'freq1_freq',
                'freq1_lambda',
                'freq1_rel_phase2',
                'freq1_rel_phase3',
                'freq1_rel_phase4',
                'freq1_signif',
                'freq2_amplitude1',
                'freq2_amplitude2',
                'freq2_amplitude3',
                'freq2_amplitude4',
                'freq2_freq',
                'freq2_rel_phase2',
                'freq2_rel_phase3',
                'freq2_rel_phase4',
                'freq3_amplitude1',
                'freq3_amplitude2',
                'freq3_amplitude3',
                'freq3_amplitude4',
                'freq3_freq',
                'freq3_rel_phase2',
                'freq3_rel_phase3',
                'freq3_rel_phase4',
                'freq_amplitude_ratio_21',
                'freq_amplitude_ratio_31',
                'freq_frequency_ratio_21',
                'freq_frequency_ratio_31',
                'freq_model_max_delta_mags',
                'freq_model_min_delta_mags',
                'freq_model_phi1_phi2',
                'freq_n_alias',
                'freq_signif_ratio_21',
                'freq_signif_ratio_31',
                'freq_varrat',
                'freq_y_offset',
                'linear_trend',
                'medperc90_2p_p',
                'p2p_scatter_2praw',
                'p2p_scatter_over_mad',
                'p2p_scatter_pfold_over_mad',
                'p2p_ssqr_diff_over_var',
                'scatter_res_raw'
               ]
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    string = "insert into " + tName + """ values (
            ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
            ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
    
    for oid in np.unique(lc['oid']):
        mask = np.where(lc['oid'] == oid)
        
        fset = featurize.featurize_time_series(lc[mask]['obsmjd'], lc[mask]['mag_autocorr'], lc[mask]['magerr_auto'],
                                        meta_features = {'oid': str(oid)}, features_to_use = feats_to_use)
        
        try: 
            cur.execute(string, fset.get_values()[0])
        except Exception as e:
            print(e)
            

In [7]:
tName = 'feats2'
string = "insert into " + tName + """ values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""

cur.execute(string, fset.get_values()[0])

In [ ]:
cur.execute("""delete from feats2""")
cur.fetchall()

In [ ]:
saving(curve, 'featurestest.db', 'feats2')

In [8]:
cur.execute("""select oid from feats2""")
cur.fetchall()

[(0.5095000000000001,)]

In [10]:
cur.execute("""select * from feats2""")
cur.fetchall()

[(0.5095000000000001,
  0.3007377525228737,
  0.43400204490596406,
  0.5932806752901907,
  0.6868684512638921,
  0.8554775768409026,
  162.62135931079118,
  15.606,
  15.325,
  0.17400000000000126,
  14.587,
  0.9733312795365007,
  0.1506849315068493,
  0.1917808219178082,
  0.773066126770398,
  103.49729008134356,
  3.9235854914604884,
  0.09425302242639545,
  -0.5006398841072615,
  0.2562918904699493,
  1.7244018225440325,
  1.0204204978189872,
  15.250295565036485,
  -3.1087886844331463,
  2.0278450170523588,
  0.27506666234997945,
  0.13818653432855071,
  0.08108273489278138,
  0.044763800692900306,
  1.5075748677663232,
  0.08428357891711286,
  2.4738686730162778,
  -0.9016153517269,
  2.0184028253427164,
  6.585500643191713,
  0.016320055479104197,
  0.0008227371290089935,
  0.0004426308733244697,
  7.375736232152197e-05,
  28.568167249767125,
  -1.9478950330781715,
  -2.2472996197295507,
  1.5825818539160765,
  0.01228776978288731,
  0.0007294435144718135,
  0.000118454940871830